In [ ]:
library(fpp3)

In [ ]:
gdppc <- global_economy %>%
    mutate(GDP_per_capita = GDP / Population)

In [ ]:
gdppc %>%
    filter(Country == "Sweden") %>%
    autoplot(GDP_per_capita) %>%
    labs(y = "$US", title = "GDP per capita for Sweden")

In [ ]:
fit <- gdppc %>%
    model(trend_model = TSLM(GDP_per_capita ~ trend()))

In [ ]:
fit %>%
    forecast(h = "3 years") %>%
    filter(Country == "Sweden") %>%
    autoplot(gdppc) + 
    labs(y = "$US", title = "GDP per capita for Sweden")

In [ ]:
bricks <- aus_production %>%
    filter_index("1970 Q1" ~ "2004 Q4")

In [ ]:
bricks %>% model(NAIVE(Bricks)) %>%
    forecast(h = "3 years") %>%
    autoplot(bricks) + 
    labs(y = "Bricks", title = "I love bricks")

In [ ]:
# Reindex with trading days
google_stock <- gafa_stock %>%
  filter(Symbol == "GOOG", year(Date) >= 2015) %>%
  mutate(day = row_number()) %>%
  update_tsibble(index = day, regular = TRUE)
# Filter the year of interest
google_2015 <- google_stock %>% filter(year(Date) == 2015)

autoplot(google_2015, Close) +
    labs(y = "$US", title = "Google daily closing price in 2015")

In [ ]:
google_2015 %>%
    model(NAIVE(Close)) %>%
    gg_tsresiduals()

In [ ]:
aug <- google_2015 %>%
    model(NAIVE(Close)) %>%
    augment()

aug %>% features(.innov, ljung_box, lag = 10, dof = 0)

In [ ]:
google_2015 %>%
    model(NAIVE(Close)) %>%
    forecast(h = 10) %>%
    autoplot(google_2015)

In [ ]:
fit <- google_2015 %>%
    model(NAIVE(Close))

sim <- fit %>% generate(h = 30, times = 5, bootstrap = TRUE)

google_2015 %>%
    ggplot(aes(day)) +
    geom_line(aes(y = Close)) +
    geom_line(aes(y = .sim, color = as.factor(.rep)), data = sim) +
    guides(col = FALSE)

In [ ]:
google_2015 %>%
    model(NAIVE(Close)) %>%
    forecast(h = 30, bootstrap = TRUE) %>%
    autoplot(google_2015)

In [ ]:
us_retail_employment <- us_employment %>%
    filter(year(Month) >= 1990, Title == "Retail Trade")

dcmp <- us_retail_employment %>%
    model(stlf = decomposition_model(
        STL(Employed ~ trend(window = 7), robust = TRUE),
        NAIVE(season_adjust)
    ))
      
dcmp %>%
    forecast() %>%
    autoplot(us_retail_employment)

In [ ]:
dcmp %>% gg_tsresiduals()